In [1]:
import nest_asyncio

nest_asyncio.apply()

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

import logging
import sys
from IPython.display import Markdown

In [2]:
from llama_index import ListIndex, LLMPredictor
from langchain import OpenAI
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.schema import MetadataMode

In [3]:
llm_predictor = LLMPredictor(
    llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=512)
)

In [4]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    MetadataFeatureExtractor,
)
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

In [5]:
class CustomExtractor(MetadataFeatureExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": node.metadata["document_title"]
                + "\n"
                + node.metadata["excerpt_keywords"]
            }
            for node in nodes
        ]
        return metadata_list

In [6]:
metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=5),
        QuestionsAnsweredExtractor(questions=3),
        SummaryExtractor(summaries=["prev", "self"]),
        KeywordExtractor(keywords=10),
        CustomExtractor()
    ],
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [7]:
from llama_index import SimpleDirectoryReader, DocumentSummaryIndex

In [12]:
from pathlib import Path

documents_ho3 = SimpleDirectoryReader(input_files=['./data/policy_docs/HO3_sample.pdf']).load_data()

In [13]:
nodes_ho3 = node_parser.get_nodes_from_documents(documents_ho3)

In [15]:
nodes_ho3[2].metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'document_title': 'Insurance Coverage for Aircraft, Hovercraft, Motor Vehicles, Watercraft, Animals, and Residences.',
 'questions_this_excerpt_can_answer': '\n\n1. What types of services are excluded from insurance coverage?\n2. Who is considered an "employee" under the terms of the insurance policy?\n3. What are the age requirements for a student to be considered an "insured" under the policy?',
 'prev_section_summary': '\nThis section defines the terms "aircraft", "hovercraft", "watercraft", "motor vehicle", "bodily injury", "business", "employee", and "insured". It outlines the activities that constitute a business, and the types of employees that are covered by the policy. It also states that residents of the insured\'s household are also covered.',
 'section_summary': '\nThis section defines the terms "employee" and "insured" in the context of home day care services. An "employee" is defined as an employee of an "insured" or an

In [16]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import FaissVectorStore
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

In [18]:
print(
    "LLM sees:\n",
    (nodes_ho3)[0].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 [Excerpt from document]
page_label: 1
file_name: HO3_sample.pdf
document_title: Insurance Coverage for Aircraft, Hovercraft, Motor Vehicles, Watercraft, Animals, and Residences.
section_summary: 
This section of the policy outlines the agreement between the company providing the insurance and the named insured. It also defines certain words and phrases used in the policy, such as Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability. These definitions include the liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use, loading, or unloading of the vehicle or craft, as well as the entrustment of the vehicle or craft to any person, the failure to supervise or negligent supervision of any person involving the vehicle or craft, and the vicarious liability for the actions of a child or minor involving the vehicle or craft.
excerpt_keywords:  Homeowners, Insurance, Agreement, Definition

In [21]:
index = VectorStoreIndex(nodes=nodes_ho3)
engine = index.as_query_engine(similarity_top_k=10)

In [22]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, node_parser=node_parser
)
question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [23]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="homeowners_policy_documents",
                description="insurance policy contract detailing coverage provisions.",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=True,
)

In [24]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    Include a bullet point list of citations in your answer.
    """
)
print(response.response)

Generated 2 sub questions.
[sec_filing_documents] Q: By first identifying and quoting the most relevant sources, what is the policy on water damage for dwelling damage?
[sec_filing_documents] Q: By first identifying and quoting the most relevant sources, what is the policy on water damage for personal property damage?
[sec_filing_documents] A: 

The policy on water damage for personal property damage is outlined in Section I – Exclusions A.3. Water Damage, which states: "We do not cover loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through this opening." Additionally, Section C. Damage To Property Of Others states: "We will pay, at replacement cost, up to $1,000 per 'occurrence' for 'property damage' to property of others caused by an 'insured', subject to the limit of liability under the policy and the

In [30]:
Markdown(f"{response.response}")

The policy on water damage for dwelling damage is outlined in the HO 00 03 10 00 document, page 9, which states: "We insure for direct physical loss to the property described in Coverage C caused by any of the following perils unless the loss is excluded in Section I – Exclusions. 1. Fire Or Lightning 2. Windstorm Or Hail. This peril includes loss to watercraft of all types and their trailers, furnishings, equipment, and outboard engines or motors, only while inside a fully enclosed building. This peril does not include loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through this opening. However, we do insure for loss caused by mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within: (a)A plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the "residence premises"; or (b)A storm drain, or water, steam or sewer pipes, off the "residence premises". For purposes of this provision, a plumbing system or household appliance does not include a sump, sump pump or related equipment or a roof drain, gutter, downspout or similar fixtures or equipment; or (6)Any of the following: (a)Wear and tear, marring, deterioration; (b)Mechanical breakdown, latent defect, inherent vice, or any quality in property that causes it to damage or destroy itself; (c)Smog, rust or other corrosion, or dry rot; (d)Smoke from agricultural smudging or industrial operations; (e)Discharge, dispersal, seepage, migration, release or escape of pollutants unless the discharge, dispersal, seepage, migration, release or escape is itself caused by a Peril Insured Against named under Coverage C. Pollutants means any solid, liquid, gaseous or thermal irritant or contaminant, including smoke, vapor, soot, fumes, acids, alkalis, chemicals and waste. Waste includes materials to be recycled, reconditioned or flues, pipes, wiring and drains. We will pay no more than the actual cash value of the damage until actual repair or replacement is complete. Once actual repair or replacement is complete, we will settle the loss as noted in 2.a. and b. above. However, if the cost to repair or replace the damage is both: (1)Less than 5% of the amount of insurance in this policy on the building; and (2)Less than $2,500; we will settle the loss as noted in 2.a. and b. above whether or not actual repair or replacement is complete. You may disregard the replacement cost loss settlement provisions and make claim under this policy for loss to buildings on an actual cash value basis. You may then make claim for any additional liability according to the provisions of this Condition C. Loss Settlement, provided you notify us of your intent to do so within 180 days after the unloading of aircraft, hovercraft, watercraft or motor vehicles. Additionally, the policy does not cover loss resulting from broken glass or safety glazing material, or from a dwelling that has been vacant for more than 60 consecutive days. It also states that the policy will pay up to $2,500 for appliances, carpeting, and other household furnishings in each apartment on the "residence premises" regularly rented or held for rental to others. Finally, it states that the policy may be used to cover up to 10% of the limit of liability for increased costs due to the enforcement of any ordinance or law (HO 00 03 10 00, page 9).

The policy on water damage for personal property damage is outlined in Section I – Exclusions A.3. Water Damage, which states: "We do not cover loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through this opening" (HO 00 03 10 00, Section I – Exclusions A.3. Water Damage). Additionally, Section C. Damage To Property Of Others states: "We will pay, at replacement cost, up to $1,000 per 'occurrence' for 'property damage' to property of others caused by an 'insured', subject to the limit of liability under the policy and the replacement cost of the damaged part of the building. We will pay no more than the actual cash value of the damage until actual repair or replacement is complete. However, if the cost to repair or replace the damage is both less than 5% of the amount of insurance in this policy on the building and less than $2,500, we will settle the loss as noted in 2.a. and b. above whether or not actual repair or replacement is complete" (HO 00 03 10 00, Section C. Damage To Property Of Others). Furthermore, our limit of liability for personal property usually located at an "insured's" residence, other than the "residence premises", is 10% of the limit of liability for Coverage C, or $1,000, whichever is greater. However, this limitation does not apply to personal property moved from the "residence premises" because it is being repaired, renovated or rebuilt and is not fit to live in or store property in, or in a newly acquired principal residence for 30 days from the time you begin to move the property there. The special limit for each category is the total limit for the coverage (HO 00 03 10 00, Section C. Damage To Property Of Others). Additionally, Section I of the policy excludes losses caused by ordinance or law, earth movement (including earthquakes, land shock waves, and tremors), nuclear hazard, and intentional loss. It also excludes losses caused by war, government action, and pollution, including losses caused by landslides, mudslides, subsidence, sinkholes, water damage, power failure, neglect, and war. In the event of a loss caused by water damage or power failure, the policy covers property of the following types: personal property, awnings, carpeting, household appliances, outdoor antennas and outdoor equipment, whether or not attached to the structure. The insured must provide proof of loss, including showing the damaged property, providing records and documents, submitting to an examination under oath, and sending a signed, sworn proof of loss. Covered property losses are settled as follows: the terms "cost to repair or replace" and "replacement cost" do not include the increased costs incurred to comply with the enforcement of any ordinance or law, except to the extent that coverage for these increased costs is provided in E.11. Ordinance Or Law under Section I – Property Coverages (HO 00 03 10 00, Section I). 

In summary, the policy on water damage for dwelling damage covers direct physical loss to the property described in Coverage C caused by fire or lightning, windstorm or hail, and mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within. The policy does not cover loss resulting from broken glass or safety glazing material, or from a dwelling that has been vacant for more than 60 consecutive days. It also states that the policy will pay up to $2,500 for appliances, carpeting, and other household furnishings in each apartment on the "residence premises" regularly rented or held for rental to others. Finally, it states that the policy may be used to cover up to 10% of the limit of liability for increased costs due to the enforcement of any ordinance or law. The policy on water damage for personal property damage does not cover loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through this opening. Additionally, the policy will pay, at replacement cost, up to $1,000 per 'occurrence' for 'property damage' to property of others caused by an 'insured', subject to the limit of liability under the policy and the replacement cost of the damaged part of the building. Furthermore, the limit of liability for personal property usually located at an "insured's" residence, other than the "residence premises", is 10% of the limit of liability for Coverage C, or $1,000, whichever is greater. (HO 00 03 10 00).

In [32]:
response = final_engine.query(
    """
    What are the differences in how the policy addresses water damage for dwelling damage and personal property damage?
    Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer.
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\

In [33]:
Markdown(f"{response.response}")

Answer: The policy outlines different coverage for water damage to dwellings and personal property depending on the source of the water. For water damage to dwellings, the policy covers ensuing loss to property described in Coverages A and B, not precluded by any other provision in the policy, including loss to buildings covered under Coverage A or B at replacement cost without deduction for depreciation, subject to the limit of liability under the policy and the replacement cost of the damaged part of the building. However, the policy does not cover loss to the system or appliance from which the water or steam escaped, or to sump, sump pump, or related equipment, or roof drain, gutter, downspout, or similar fixtures or equipment. For water damage caused by freezing, thawing, pressure or weight of water or ice, the policy covers loss to a fence, pavement, patio or swimming pool; footing, foundation, bulkhead, wall, or any other structure or device that supports all or part of a building, or other structure; retaining wall or bulkhead that does not support all or part of a building or other structure; or pier, wharf or dock. For water damage caused by mold, fungus or wet rot, the policy covers loss hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the "residence premises", or a storm drain, or water, steam or sewer pipes, off the "residence premises". The policy also excludes increased costs incurred to comply with the enforcement of any ordinance or law, except to the extent that coverage for these increased costs is provided in the policy. 

For water damage to personal property, the policy covers personal property owned or used by an "insured" while it is anywhere in the world, with a limit of liability not more than 10% of the limit of liability that applies to Coverage A. The policy also outlines coverage for water damage to property contained in a building caused by rain, snow, sleet, sand or dust if the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters the building. This coverage is provided under Section I – Exclusion A.3. Water Damage, Paragraphs a., c., and d. It excludes losses caused by flooding, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment; or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure. Direct loss by fire, explosion or theft resulting from water damage is covered. The policy also covers loss to watercraft of all types and their trailers, furnishings, equipment, and outboard engines or motors, only while inside a fully enclosed building. 

Sources: 
Section I – Exclusion A.3. Water Damage, Paragraphs a., c., and d. 
Section I – Property Coverages, Peril 2. Windstorm or Hail 
Coverages A and B 
Section I – Exclusion A.3. Water Damage, Paragraphs a., c., and d. 
Section I – Property Coverages, Peril 2. Windstorm or Hail

In [48]:
index.docstore.get_document(doc_id="90fd4e93-3af6-4bf6-a91e-5d561214d7cc").metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'document_title': 'Insurance Coverage for Aircraft, Hovercraft, Motor Vehicles, Watercraft, Animals, and Residences.',
 'questions_this_excerpt_can_answer': "\n\n1. What types of liability are covered under this policy?\n2. What types of vehicles and crafts are included in the definition of Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability?\n3. What is the scope of an insured's liability for the actions of a child or minor involving a vehicle or craft?",
 'section_summary': '\nThis section of the policy outlines the agreement between the company providing the insurance and the named insured. It also defines certain words and phrases used in the policy, such as Aircraft Liability, Hovercraft Liability, Motor Vehicle Liability, and Watercraft Liability. These definitions include the liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use

In [50]:
print(index.docstore.get_document(doc_id="90fd4e93-3af6-4bf6-a91e-5d561214d7cc").get_content())

[Excerpt from document]

Excerpt:
-----
HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(

In [52]:
index.set_index_id("ho3_custom_metadata")
index.storage_context.persist(persist_dir="./_policy_index_metadatas")

DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/docstore.json
open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/docstore.json
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/index_store.json
open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/index_store.json
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/vector_store.json
open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/vector_store.json
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/graph_store.json
open file: c:/Users/pdoub/Desktop/python_projects/liberty-gpt/_policy_index_metadatas/graph_store.json


In [11]:
from IPython.display import Markdown
from llama_index import (ServiceContext, 
                         StorageContext,
                         LLMPredictor,
                         load_index_from_storage)
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata
from langchain.chat_models import ChatOpenAI

def get_llm_predictor(temperature=0):
    return LLMPredictor(ChatOpenAI(temperature=temperature, model="gpt-3.5-turbo"))

service_context = ServiceContext.from_defaults(llm_predictor=get_llm_predictor())

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./_policy_index_metadatas"),
    )

engine = index.as_query_engine(similarity_top_k=10)

question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="homeowners_policy_documents",
                description="insurance policy contract detailing coverage provisions.",
            ),
        )
    ],
    service_context=service_context,
    use_async=False,
)

In [17]:
from IPython.display import Markdown

In [12]:
query = "What are the differences in how the policy addresses water damage for dwelling damage and personal property damage?"

response = final_engine.query(query)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Generated 2 sub questions.
[homeowners_policy_documents] Q: How does the policy address water damage for dwelling damage?
[homeowners_policy_documents] A: 

The policy covers direct physical loss to the property described in Coverage C caused by water, freezing, theft, vandalism, and mold, fungus, or wet rot. For water damage, the policy specifies that water supply and heat must be maintained for coverage to apply, and that losses caused by mold, fungus, or wet rot must result from an accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system. The policy also covers ensuing loss to property described in Coverages A and B, not precluded by any other provision in the policy, caused by fire or lightning, windstorm or hail, and rain, snow, sleet, sand, or dust if the direct force of wind or hail damages the building. The coverage also includes accidental discharge or overflow of water or steam from wit

In [33]:
print(response.source_nodes[0].node.get_text())

Sub question: How does the policy address water damage for dwelling damage?
Response: 

The policy covers direct physical loss to the property described in Coverage C caused by water, freezing, theft, vandalism, and mold, fungus, or wet rot. For water damage, the policy specifies that water supply and heat must be maintained for coverage to apply, and that losses caused by mold, fungus, or wet rot must result from an accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system. The policy also covers ensuing loss to property described in Coverages A and B, not precluded by any other provision in the policy, caused by fire or lightning, windstorm or hail, and rain, snow, sleet, sand, or dust if the direct force of wind or hail damages the building. The coverage also includes accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protecti

In [32]:
Markdown(f"{response}")

The policy addresses water damage for dwelling damage by covering direct physical loss to the property described in Coverage C caused by water, freezing, theft, vandalism, and mold, fungus, or wet rot. It specifies that water supply and heat must be maintained for coverage to apply, and that losses caused by mold, fungus, or wet rot must result from an accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system. The policy also covers ensuing loss to property described in Coverages A and B caused by fire or lightning, windstorm or hail, and rain, snow, sleet, sand, or dust if the direct force of wind or hail damages the building. It includes coverage for accidental discharge or overflow of water or steam, sudden and accidental tearing apart, cracking, burning or bulging, and freezing of plumbing, heating, air conditioning or automatic fire protective sprinkler systems, and household appliances. However, it excludes loss to the system or appliance from which the water or steam escaped, or to sump, sump pump, or related equipment, or roof drain, gutter, downspout, or similar fixtures or equipment.

On the other hand, the policy addresses water damage for personal property damage by covering direct physical loss to the property described in Coverage C caused by water, including loss to watercraft of all types and their trailers, furnishings, equipment, and outboard engines or motors, only while inside a fully enclosed building. It does not cover loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through the opening. Similar to dwelling damage, it excludes loss to the system or appliance from which the water or steam escaped, or to sump, sump pump, or related equipment, or roof drain, gutter, downspout, or similar fixtures or equipment. In the event of a loss caused by water damage, the coverage is settled at actual cash value at the time of loss, but not more than the amount required to repair or replace. The policy also includes provisions for determining the amount of insurance required for replacement cost coverage and settlement of covered property losses.

In [22]:
Markdown(f"{response.source_nodes[0].node.text}")

Sub question: How does the policy address water damage for dwelling damage?
Response: 

The policy covers direct physical loss to the property described in Coverage C caused by water, freezing, theft, vandalism, and mold, fungus, or wet rot. For water damage, the policy specifies that water supply and heat must be maintained for coverage to apply, and that losses caused by mold, fungus, or wet rot must result from an accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system. The policy also covers ensuing loss to property described in Coverages A and B, not precluded by any other provision in the policy, caused by fire or lightning, windstorm or hail, and rain, snow, sleet, sand, or dust if the direct force of wind or hail damages the building. The coverage also includes accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system, as well as sudden and accidental tearing apart, cracking, burning or bulging of a steam or hot water heating system, an air conditioning or automatic fire protective sprinkler system, or an appliance for heating water, and freezing of a plumbing, heating, air conditioning or automatic fire protective sprinkler system or of a household appliance, but only if reasonable care has been used. The coverage does not include loss to the system or appliance from which the water or steam escaped, or to sump, sump pump, or related equipment, or roof drain, gutter, downspout, or similar fixtures or equipment. Additionally, the policy covers losses from aircraft, hovercraft, motor vehicles, watercraft, animals, and residences, as well as losses from fire, lightning, windstorm, hail, explosion, smoke, aircraft, vehicles, riot, civil commotion, vandalism, leakage from plumbing, heating, air conditioning, and sprinkler systems, falling objects, and weight of ice, snow, or sleet. However, the policy does not cover losses from the construction of a dwelling, property rented to someone other than an insured, trailers, semitrailers, campers, watercraft, and property at another residence owned by, rented to, or occupied by an insured. It also excludes losses on the "residence premises" if the dwelling has been vacant for more than 60 consecutive days immediately before the loss, except when the breakage results directly from earth movement. Key topics include accidental discharge or overflow of water or steam, sudden and accidental tearing apart, cracking, burning or bulging, and freezing. Entities include plumbing, heating, air conditioning or automatic fire protective sprinkler systems, and household appliances.

In [23]:
Markdown(f"{response.source_nodes[1].node.text}")

Sub question: How does the policy address water damage for personal property damage?
Response: 

The policy covers direct physical loss to the property described in Coverage C caused by water, including loss to watercraft of all types and their trailers, furnishings, equipment, and outboard engines or motors, only while inside a fully enclosed building. However, the policy does not cover loss to the property contained in a building caused by rain, snow, sleet, sand or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand or dust enters through the opening. The policy also does not cover loss to the system or appliance from which the water or steam escaped, or to sump, sump pump, or related equipment, or roof drain, gutter, downspout, or similar fixtures or equipment. In the event of a loss caused by water damage, the coverage is settled at actual cash value at the time of loss, but not more than the amount required to repair or replace. For buildings covered under Coverage A or B, the coverage is at replacement cost without deduction for depreciation, subject to the limit of liability under the policy and the replacement cost of the damaged part of the building. If the building is rebuilt at a new premises, the cost described in (2) above is limited to the cost which would have been incurred if the building had been built at the original premises. If, at the time of loss, the amount of insurance in this policy on the damaged building is less than 80% of the full replacement cost of the building immediately before the loss, we will pay the greater of the following amounts, but not more than the limit of liability under this policy that applies to the building: (1) The actual cash value of that part of the building damaged; or (2) That proportion of the cost to repair or replace, after application of any deductible and without deduction for depreciation, that part of the building damaged, which the total amount of insurance in this policy on the damaged building bears to 80% of the replacement cost of the building. To determine the amount of insurance required to equal 80% of the full replacement cost of the building immediately before the loss, do not include the value of: (1) Excavations, footings, foundations, piers, or any other structures or devices that support all or part of the building, which are below the undersurface of the lowest basement floor; (2) Those supports described in (1) above which are below the surface of the ground inside the foundation walls, if there is no basement; and (3) Underground flues, pipes, wiring and drains. We will pay no more than the actual cash value of the damage until actual repair or replacement is complete. Once actual repair or replacement is complete, we will settle the loss as noted in 2.a. and b. above. However, if the cost to repair or replace the damage is both: (1) Less than 5% of the amount of insurance in this policy on the building; and (2) Less than $2,500; we will settle the loss. In addition, the terms "cost to repair or replace" and "replacement cost" do not include the increased costs incurred to comply with the enforcement of any ordinance or law, except to the extent that coverage for these increased costs is provided in E.11. Ordinance Or Law under Section I – Property Coverages. Covered property losses are settled as follows: 1. Property of the following types: a. Personal property; b. Awnings, carpeting, household appliances, outdoor antennas and outdoor equipment, whether or not attached to the building; and c. Trees, shrubs and other plants will be settled at actual cash value at the time of loss.

In [58]:
test_doc = documents_ho3[0].get_text()
print(test_doc)

HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(3)Entrustment of such vehicle or craft b

In [57]:
import re
from collections import deque

class Node:
    def __init__(self, text):
        self.text = text
        self.children = []

    def add_child(self, node):
        self.children.append(node)

    def get_text(self):
        return self.text

    def get_children(self):
        return self.children

# Regular expressions to match section headers
regex_patterns = [
    (re.compile(r'^(?P<title>[A-Z\s\d–]+)$', re.MULTILINE), Node),
    (re.compile(r'^(?P<section>[A-Z]\.)', re.MULTILINE), Node),
    (re.compile(r'^(?P<subsection>\d+\.)', re.MULTILINE), Node),
    (re.compile(r'^(?P<subpoint>[a-z]\.)', re.MULTILINE), Node),
    (re.compile(r'^(?P<point>\(\d+\))', re.MULTILINE), Node),
    (re.compile(r'^(?P<subpoint>\([a-z]\))', re.MULTILINE), Node),
]

def build_tree(text):
    lines = text.split('\n')
    root = Node("")
    stack = [root]

    for line in lines:
        for pattern, NodeClass in regex_patterns:
            match = pattern.match(line)
            if match:
                node = NodeClass(line)
                stack[-1].add_child(node)
                stack.append(node)
                break
        else:
            if stack:
                stack[-1].text += '\n' + line

    return root

def extract_sections(root, char_limit):
    stack = deque([(root, '')])
    while stack:
        node, path = stack.pop()
        new_path = path + '\n' + node.get_text() if path else node.get_text()
        if len(new_path) > char_limit:
            return path
        if node.get_children():
            stack.extend((child, new_path) for child in node.get_children())
        else:
            return new_path


In [59]:
tree = build_tree(test_doc)

In [73]:
extracted_text = extract_sections(tree, 1500)

In [74]:
print(extracted_text)

HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(3)Entrustment of such vehicle or craft b

In [64]:
tree.get_children()